<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_2_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,24,heart beat for being too strong
2,62,feeling proper psycho
3,94,anxiety
4,133,night terrors


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-2.test.csv",header=None)
test.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,24,racing heart
2,56,persistent painful erection
3,57,hypertrophic scarring
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'i',
 'at',
 'patients',
 'from',
 'or',
 'this',
 'you',
 'disease',
 'be',
 'mg',
 'group',
 'increased',
 'my',
 'study',
 'mood',
 'are',
 'not',
 'treatment',
 'an',
 'me',
 'it',
 'after',
 'amp',
 'have',
 'p',
 'release',
 'gluten',
 ',',
 'disorder',
 'all',
 'we',
 'free',
 'effect',
 'withdrawal',
 'abnormal',
 'no',
 'symptoms',
 'than',
 'decreased',
 'new',
 'weight',
 'your',
 'compared',
 'out',
 'day',
 'anxiety',
 'using',
 'effects',
 'more',
 'groups',
 'time',
 'up',
 'our',
 'about',
 'between',
 'inflammatory',
 'two',
 'h',
 'blood',
 'both',
 'its',
 'like',
 'significantly',
 'acute',
 'drugs',
 'these',
 'significant',
 'had',
 'anti',
 'can',
 'but',
 'potassium',
 'skin',
 'voltaren',
 'used',
 'dose',
 'd

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.727101,4.192534,0.345847,1:05:25


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.015398,3.811608,0.383036,1:06:12


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.711154,3.768576,0.387473,1:06:43


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.678114,3.736897,0.390578,1:08:25


In [25]:
learn.save_encoder('pubmed_cadec_twadrl_mixed_fold2_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_twadrl_mixed_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.161222,6.473523,0.092857,01:10


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.195309,5.420801,0.092857,01:21


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.677585,5.250565,0.128571,02:47


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.390932,4.830639,0.178571,02:57
1,5.159063,4.709438,0.157143,02:57


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.915034,4.484533,0.185714,03:00
1,4.698413,4.359352,0.221429,02:53


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.539064,4.272354,0.228571,03:02
1,4.361889,4.242103,0.221429,02:47


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.155115,4.078471,0.264286,03:00
1,3.983797,3.962416,0.321429,02:56


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.818940,3.918817,0.307143,02:50
1,3.656406,3.811095,0.328571,03:00


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.553547,3.797604,0.335714,02:49
1,3.419807,3.750122,0.314286,03:01
2,3.251576,3.616993,0.364286,02:57
3,3.100057,3.650171,0.364286,02:47


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.925767,3.642692,0.335714,02:50
1,2.842019,3.521856,0.392857,02:54
2,2.678724,3.578883,0.371429,02:48
3,2.523904,3.551683,0.371429,02:55


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.352624,3.529103,0.378571,03:09
1,2.316777,3.573077,0.385714,02:45
2,2.143351,3.533997,0.385714,03:00
3,1.999273,3.497978,0.392857,03:00


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.882691,3.574736,0.421429,02:55
1,1.852416,3.673790,0.414286,03:04
2,1.708286,3.559109,0.400000,03:05
3,1.583490,3.601654,0.392857,03:03


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.460193,3.644497,0.407143,02:52
1,1.420687,3.681850,0.378571,02:50
2,1.337550,3.702210,0.407143,03:01
3,1.218928,3.745990,0.407143,03:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.139621,3.731466,0.392857,03:02
1,1.121308,3.793948,0.407143,03:04
2,1.039532,3.781830,0.421429,02:46
3,0.954766,3.825248,0.435714,02:55


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.867782,3.827138,0.435714,03:00
1,0.901290,3.992245,0.392857,02:50
2,0.817905,3.912463,0.421429,02:49
3,0.771158,3.927340,0.407143,02:59


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.724964,3.956065,0.414286,02:56
1,0.684359,4.079732,0.364286,03:04


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.692360,4.148520,0.392857,02:59
1,0.658867,4.114002,0.378571,02:53


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.599365,4.304648,0.407143,02:44


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.596055,4.274840,0.407143,02:56


In [53]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.562568,4.442113,0.407143,02:55


In [54]:
# save the best model

learn.save_encoder('pubmed_cadec_twadrl_mixed_fold2')

# Part three: Predict on the test dataset

In [55]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [56]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[2191, 1507, 56, 899, 978, 1942, 189, 189, 189, 189, 217, 1521, 233, 1936, 233, 108, 301, 1787, 1787, 393, 396, 418, 445, 923, 582, 908, 743, 720, 744, 744, 749, 749, 749, 887, 1349, 1099, 792, 792, 2125, 1701, 824, 1682, 265, 1184, 1833, 974, 974, 974, 978, 2168, 1019, 359, 359, 1767, 620, 1099, 224, 1940, 1099, 1136, 441, 1184, 1184, 831, 1373, 1265, 1265, 1295, 1295, 852, 1295, 1308, 1373, 1340, 1340, 1507, 1973, 2191, 1099, 1833, 947, 582, 978, 233, 1701, 1526, 1529, 1529, 2149, 978, 109, 1597, 1597, 2147, 1099, 1597, 1610, 108, 1701, 1526, 109, 1767, 1767, 1718, 1815, 1815, 1718, 1815, 1815, 1767, 1768, 809, 1787, 1787, 1787, 1787, 393, 1196, 418, 645, 2191, 1073, 2032, 1936, 418, 233, 1936, 1936, 1099, 1942, 1833, 923, 1373, 579, 1948, 418, 1099, 224, 2147, 1787, 1676, 1787, 393]
[13, 24, 56, 57, 94, 113, 189, 189, 189, 189, 217, 224, 233, 233, 233, 265, 301, 393, 393, 393, 396, 418, 445, 452, 582, 605, 609, 720, 744, 745, 749, 749, 749, 749, 755, 791, 792, 792, 813, 819,

In [57]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

60
0.4195804195804196
